# Vehicle Collision Dataset Analysis: Seattle Department of Transportation (SDOT)

### <i> Check out the complete analysis here <a href="https://medium.com/intellyticssolutions/data-analysis-beyond-plotting-of-colourful-visualisations-35886facad21">@ Medium</a></i>  

<img src="https://images.unsplash.com/photo-1575987446487-56eba08666cf?ixlib=rb-1.2.1&auto=format&fit=crop&w=750&q=80">

According to the annual United States road crash statistics by ASIRT, more than 38,000 people die every year in crashes on U.S. roadways. The U.S. traffic fatality rate is 12.4 deaths per 100,000 inhabitants. It is evident that with the increasing number of vehicles on urban and suburban roads, the cases of vehicle accidents are also increasing. This project aims to analyse vehicle collision data available in public domain provided by Seattle Geo Data (SDOT) and derive some useful insights. <br>
Data provided by the Seattle Department of Transportation (SDOT) on vehicle collisions along with its severity might be useful to derive insights and may show some pattern with the environmental factors like weather, road conditions etc. <br>
The dataset consists of 40 columns having different kinds of data like, collision severity, road conditions, number of people involved, location of collision, weather etc. Meta-data of the dataset can be viewed https://www.seattle.gov/Documents/Departments/SDOT/GIS/Collisions_OD.pdf

# Important Insights

1. 62% of accidents belong to category "Property Damage Only Collision" while 26% belong to category "Injury Collision" <br>
2. 50% of accidents tend to happen between 12:00 P.M. to 8:00 P.M.<br>
3. Between 00:00 to 04:00, cases of severity 1 ("Property damage only collision") incidents are higher than rest of the day percentage wise<br>
4. 11% of all cases in late night are due to speeding <br>
5. ~18% of collisions happening in morning and early morning are due to inattention of drivers <br>
6. Motor Vehicles tend to hit fixed objects more during late night time <br>
7. Severe accidents majorly happens at a block or an intersection <br>
8. Location "6th AVE AND JAMES ST" has abnormally high number of collisions since 2016 <br>
9. There is a sudden drop in accident cases in 2020, due to COVID-19 pandemic<br>
10. There are over 250 cases of collisions at "BATTERY ST TUNNEL"<br>
11. There are relatively more cases of alcoholic and drug influenced collision during December (Probably because of holiday season)

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap, HeatMapWithTime, MarkerCluster
from pandas_profiling import ProfileReport
import plotly.express as px
import pprint
import plotly.graph_objs as go
import matplotlib as mpl
import calendar

In [ ]:
sns.palplot(sns.color_palette("GnBu_d",10))
sns.set_palette(sns.color_palette("GnBu_d",10))

In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

# Reading the Data

In [ ]:
df = pd.read_csv("../input/seattle-sdot-collisions-data/Collisions.csv", low_memory=False, parse_dates=["INCDATE"])
df_original = df.copy()
df.head()

In [ ]:
df.shape

The dataset consists of 40 distinct columns and 220937 rows

```OBJECTID``` is just a unique value for each row. 
```INCKEY``` and ```COLDETKEY``` are unique identifier for a given incident

In [ ]:
corrMatrix = df.corr()
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(corrMatrix, annot=True, ax=ax)
plt.show()

# Data Cleaning

In [ ]:
df.drop(["INCKEY","COLDETKEY"],axis=1,inplace=True)

In [ ]:
df["UNDERINFL"].unique()

For the column ```UNDERINFL``` there are 4 unique values. Intuitively, it seems that 'Y' and '1' denotes the same thing which is the person was under influence of alcohol. Same is with 'N' and '0'. Hence, let's save the column values in systematic format

In [ ]:
def for_UNDERINFL(x):
    if x in ['Y','1']:
        return "Y"
    elif x in ['N','0']:
        return "N"
    else:
        return x
    
df["UNDERINFL"] = df["UNDERINFL"].apply(for_UNDERINFL)

In [ ]:
temp = df[["SEVERITYCODE","SEVERITYDESC"]]
temp[~(temp.duplicated())].style.hide_index()

There are 5 categories of severity. It is safe to assume that the missing ```SEVERITYCODE``` must belong to unknown category i.e. Category 0.

In [ ]:
df["SEVERITYCODE"].fillna('0', inplace=True)

In [ ]:
df["ADDRTYPE"].fillna('Not Mentioned', inplace=True)

In [ ]:
df["MONTH"] = df["INCDATE"].dt.month
df["YEAR"] = df["INCDATE"].dt.year

In [ ]:
sns.countplot(df["SEVERITYCODE"])

In [ ]:
df["SEVERITYCODE"].value_counts(normalize=True)*100

We can see that 62% of accidents belong to Category 1 which is "Property Damage Only Collision" <br>
26% belong to Category 2, "Injury Collision"

# Accidents declining YoY? Not so sure

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
sns.countplot(df["INCDATE"].dt.year,ax=ax)
ax.set_xlabel('Year', fontsize=18)
ax.tick_params(axis="x", labelsize=15)
ax.tick_params(axis="y", labelsize=15)
ax.set_ylabel('Collision Count', fontsize=18)
plt.title("Collision count YoY", fontsize=18)
# plt.savefig('./blog/1.png')

Accidents are seems to be in declining yearly, but we need some more data to state this precisely. 

# Month wise collision rate

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.countplot(df["INCDATE"].dt.month,ax=ax)
ax.set_xlabel('Months', fontsize=18)
ax.tick_params(axis="x", labelsize=15)
ax.tick_params(axis="y", labelsize=15)
ax.set_ylabel('Collision Count', fontsize=18)
plt.title("Collision count MoM", fontsize=18)
# plt.savefig('./blog/2.png')

There is no clear trend in number of collisions and month of the year

In [ ]:
df["INCTIME"] = pd.to_datetime(df["INCDTTM"])
for idx, dt in enumerate(df["INCDTTM"]):
    if ':' not in dt:
        df["INCTIME"][idx] = np.nan

b = [0,4,8,12,16,20,24]
l = ['Late Night', 'Early Morning','Morning','Noon','Eve','Night']
df["TIMEOFDAY"] = pd.cut(df["INCTIME"].dt.hour, bins=b, labels=l, include_lowest=True)

In [ ]:
1 - (df["TIMEOFDAY"].isnull().sum() / len(df))

We have time of accident for ~76% of data. In order to analyse timewise, splitting the data into 6 categories viz.  'Late Night', 'Early Morning', 'Morning', 'Noon', 'Eve', 'Night'

In [ ]:
pd.DataFrame({'TIME' : ['00:00 to 04:00', '04:00 to 08:00', '08:00 to 12:00', '12:00 to 16:00', '16:00 to 20:00', '20:00 to 00:00']},
             index=['Late Night', 'Early Morning','Morning','Noon','Eve','Night'])

In [ ]:
df_TOD = pd.DataFrame(df['OBJECTID'].groupby(df['TIMEOFDAY'].astype('object')).count())
df_TOD = df_TOD.sort_values(by='OBJECTID',ascending=False)
df_TOD["cumpercentage"] = df_TOD["OBJECTID"].cumsum()/df_TOD["OBJECTID"].sum()*100


fig, ax = plt.subplots(figsize=(10,5))
ax2 = ax.twinx()
ax.bar(df_TOD.index, df_TOD["OBJECTID"])
ax2.plot(df_TOD.index, df_TOD["cumpercentage"], color="C9", marker="D", ms=7)
ax2.yaxis.set_major_formatter(mpl.ticker.PercentFormatter())

ax.tick_params(axis="y", colors="C0")
ax2.tick_params(axis="y", colors="C1")

plt.title("Pareto Diagram: Vehicle collisions on Time", fontsize=18)

# plt.savefig('./blog/4.png')
plt.show()

From the pareto diagram, we can see that about 50% of accidents tend to happen during the day time i.e. between 12:00 P.M. to 8:00 P.M.

In [ ]:
pd.DataFrame(df['OBJECTID'].groupby(df['TIMEOFDAY']).count())

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.countplot(df["TIMEOFDAY"], hue=df['SEVERITYDESC'],ax=ax)

In [ ]:
pd.crosstab(df.TIMEOFDAY, df.SEVERITYCODE)

In [ ]:
fig, ax = plt.subplots(figsize=(30,20))
df_temp = df.copy()
df_temp = df_temp[df_temp["SEVERITYCODE"]!='0']
(pd.crosstab(df_temp.TIMEOFDAY, df_temp.SEVERITYDESC).apply(lambda r: r/r.sum(), axis=1)*100).plot.pie(subplots=True, ax=ax, autopct='%1.0f%%', pctdistance=0.5,labeldistance=1.2, radius=1.2, legend=None)
# plt.savefig('./blog/5.png')

There is no clear relation between severity of accidents and time of the day. During Late Night, cases of severity 1 ("Property damage only collision") incidents are higher than rest of the day percentage wise.<br>
3 of 4 accidents happening during late night are just doing property damage

In [ ]:
(pd.crosstab(df.TIMEOFDAY, df.SEVERITYDESC).apply(lambda r: r/r.sum(), axis=1)*100)

# Speeding

In [ ]:
sns.set_palette(sns.color_palette("tab10",10))
speed = df["SPEEDING"]
speed = speed.fillna("N")
speed.value_counts(normalize=True).plot.pie(subplots=True, autopct='%1.0f%%')
# plt.savefig('./blog/6.png')

In [ ]:
((df[df["SPEEDING"]=="Y"].groupby("SEVERITYDESC")["OBJECTID"].count().sort_values(ascending=False) / df[df["SPEEDING"]=="Y"].groupby("SEVERITYCODE")["OBJECTID"].count().sum())*100).plot.pie(subplots=True,figsize=(8,8), autopct='%1.0f%%' , title="Distribution of Speeding Collisions by Severity")
# plt.savefig('./blog/7.png')

In [ ]:
(df[df["SPEEDING"].isnull()]["SEVERITYDESC"].value_counts(normalize=True)*100).plot.pie(subplots=True,figsize=(8,8), autopct='%1.0f%%', title="Distribution of Non Speeding Collisions by Severity")
# plt.savefig('./blog/8.png')

We can see percentage of <br>
Injury Collision <b>increases</b> from 26.52% to 36.3% <br>
Serious Injury Collision <b>increases</b> from 1.39% to 3.92%  and<br>
Fatality Collision <b>increases</b> from 0.15% to 0.89%  <br>
because of speeding. While the percentage of Property Damage Only Collision <b>decreases</b> from 62.14% to 58.87%

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.countplot(df["TIMEOFDAY"], hue=df['SPEEDING'],ax=ax)
ax.set_xlabel('Day Time', fontsize=18)
ax.tick_params(axis="x", labelsize=12)
ax.tick_params(axis="y", labelsize=12)
ax.set_ylabel('Collision Count', fontsize=18)
plt.title("Collision count by Day Time", fontsize=18)
for p in ax.patches:
             ax.annotate("%.0f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
                 ha='center', va='center', fontsize=11, color='gray', xytext=(0, 20),
                 textcoords='offset points')
# plt.savefig('./blog/9.png')

In [ ]:
sns.set_palette(sns.color_palette("Oranges_r",10))
fig, ax = plt.subplots(figsize=(10,5))
sns.barplot(y=(df.groupby('TIMEOFDAY')['SPEEDING'].apply(lambda x: x[x == 'Y'].count()) /df.groupby('TIMEOFDAY')['OBJECTID'].count())*100, x=list(df.groupby('TIMEOFDAY').groups.keys()),ax=ax)
ax.set_xlabel('Day Time', fontsize=18)
ax.tick_params(axis="x", labelsize=12)
ax.tick_params(axis="y", labelsize=12)
ax.set_ylabel('Collision Count %', fontsize=18)
plt.title("Collision count by Day Time", fontsize=18)
for p in ax.patches:
             ax.annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
                 ha='center', va='center', fontsize=11, color='gray', xytext=(0, 20),
                 textcoords='offset points')
# plt.savefig('./blog/10.png')

11% of all cases in late night are due to speeding

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.set_palette(sns.color_palette("tab10_r",10))
df[(df["SPEEDING"]=="Y")].groupby("SDOT_COLDESC")["OBJECTID"].count().sort_values(ascending=False).head(10).plot.barh(ax=ax)
ax.set_xlabel('Collision Count', fontsize=18)
ax.tick_params(axis="x", labelsize=12)
ax.tick_params(axis="y", labelsize=12)
ax.set_ylabel('Collision Type', fontsize=18)
plt.title("Collision type by count of Speeding collisions", fontsize=18)
# plt.savefig('./blog/11.png')

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.set_palette(sns.color_palette("coolwarm_r",10))
df[df["SPEEDING"]=="Y"].groupby("LOCATION")["OBJECTID"].count().sort_values(ascending=False).head(5).plot.barh() 
ax.set_xlabel('Collision Count', fontsize=18)
ax.tick_params(axis="x", labelsize=12)
ax.tick_params(axis="y", labelsize=12)
ax.set_ylabel('Location', fontsize=18)
plt.title("Location by count of Speeding collisions", fontsize=18)
# plt.savefig('./blog/12.png')

There are over 250 cases of collisions at "BATTERY ST TUNNEL"

In [ ]:
m = folium.Map(location=[df["Y"].mean(), df["X"].mean()], zoom_start=12)
folium.Marker((47.69034175,-122.3290808),popup="MOST SPEEDING COLLISIONS: BATTERY ST TUNNEL").add_to(m)    
m

<img src="https://bridgehunter.com/photos/19/72/197274-M.jpg"> </img>

Battery Tunnel, Seattle. (img source: https://bridgehunter.com/photos/19/72/197274-M.jpg)

# Collisions due to inattention

In [ ]:
sns.set_palette(sns.color_palette("coolwarm",10))
fig, ax = plt.subplots(figsize=(10,5))
sns.countplot(df["TIMEOFDAY"], hue=df['INATTENTIONIND'],ax=ax)
ax.set_xlabel('Day Time', fontsize=18)
ax.tick_params(axis="x", labelsize=12)
ax.tick_params(axis="y", labelsize=12)
ax.set_ylabel('Collision Count', fontsize=18)
plt.title("Collision count by Day time for Inattentive driving collisions", fontsize=18)
# plt.savefig('./blog/14.png')

In [ ]:
sns.set_palette(sns.color_palette("bone",10))
fig, ax = plt.subplots(figsize=(10,5))
sns.barplot(y=((df.groupby('TIMEOFDAY')['INATTENTIONIND'].apply(lambda x: x[x == 'Y'].count()) /df.groupby('TIMEOFDAY')['OBJECTID'].count())*100).sort_values(), x=list(df.groupby('TIMEOFDAY').groups.keys()),ax=ax)
for p in ax.patches:
             ax.annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
                 ha='center', va='center', fontsize=11, color='gray', xytext=(0, 5),
                 textcoords='offset points')
ax.set_xlabel('Day Time', fontsize=18)
ax.tick_params(axis="x", labelsize=12)
ax.tick_params(axis="y", labelsize=12)
ax.set_ylabel('Collision Count %', fontsize=18)
plt.title("% of Collision count by Day time for Inattentive driving collisions", fontsize=18)
# plt.savefig('./blog/15.png')

~18% of collisions happening in morning and early morning are due to inattention of drivers

In [ ]:
sns.set_palette(sns.color_palette("brg",10))
fig, ax = plt.subplots(figsize=(10,5))
(df[df["INATTENTIONIND"]=="Y"].groupby("COLLISIONTYPE")["OBJECTID"].count().sort_values()).plot.barh()
ax.set_xlabel('Collision Count', fontsize=18)
ax.tick_params(axis="x", labelsize=12)
ax.tick_params(axis="y", labelsize=12)
ax.set_ylabel('Collision type', fontsize=18)
plt.title("Collision type by count for Inattentive driving collisions", fontsize=18)
# plt.savefig('./blog/16.png')

In [ ]:
(df[df["INATTENTIONIND"]=="Y"].groupby("COLLISIONTYPE")["OBJECTID"].count().sort_values(ascending=False))[0]/(df[df["INATTENTIONIND"]=="Y"].groupby("COLLISIONTYPE")["OBJECTID"].count().sum()) * 100

We noticed that 35% of collisions happening because of inattention are rear ended collisions.

# Under influence of alcohol or drugs

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.set_palette(sns.color_palette("GnBu",2))
sns.countplot(df["TIMEOFDAY"], hue=df['UNDERINFL'],ax=ax)
ax.set_xlabel('Day Time', fontsize=18)
ax.tick_params(axis="x", labelsize=12)
ax.tick_params(axis="y", labelsize=12)
ax.set_ylabel('Collision Count', fontsize=18)
plt.title("Collision count by Day time for under influenced driving collisions", fontsize=18)
# plt.savefig('./blog/17.png')

In [ ]:
pd.crosstab(df.TIMEOFDAY, df.UNDERINFL).apply(lambda x: (x/x.sum())*100, axis=0)

Of total alcoholic and drug cases, ~64% cases happen during Night and Late Night

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
df[df["UNDERINFL"]=="Y"].groupby("MONTH")["OBJECTID"].count().plot(ax=ax)
ax.set_xlabel('Month', fontsize=18)
ax.tick_params(axis="x", labelsize=12)
ax.tick_params(axis="y", labelsize=12)
ax.set_ylabel('Collision Count', fontsize=18)
plt.title("Collision count by Month for under influenced driving collisions", fontsize=18)
# plt.savefig('./blog/19.png')

We can see there are relatively more cases of alcoholic and drug influenced collision during December (Probably because of holiday season)

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
df[df["UNDERINFL"]=="Y"].groupby("LOCATION")["OBJECTID"].count().sort_values(ascending=False).head(10).plot.barh(ax=ax)
ax.set_xlabel('Collision Count', fontsize=18)
ax.tick_params(axis="x", labelsize=12)
ax.tick_params(axis="y", labelsize=12)
ax.set_ylabel('Location', fontsize=18)
plt.title("Location by Collision count for under influenced driving collisions", fontsize=18)
# plt.savefig('./blog/20.png')

"ALASKAN VI SB BETWEEN COLUMBIA ST ON RP AND ALASKAN WY VI SB EFR OFF RP" has greatest number of cases of underinfluence collisions

# Examining collision type

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(35,13))
sns.set_palette(sns.color_palette("winter"))
col_type = list(df['COLLISIONTYPE'].unique())
(col_type).remove(np.nan)

for i in range(0,2):
    for j in range(0,5):
        sns.countplot(df[df['COLLISIONTYPE'] == col_type[0]]['TIMEOFDAY'], ax=ax[i,j])
        ax[i,j].set_title(col_type[0])
        col_type.pop(0)
# plt.savefig('./blog/21.png')

In [ ]:
pd.crosstab(df.COLLISIONTYPE, df.TIMEOFDAY).apply(lambda r: (r/r.sum())*100, axis=0)

People are morelikely to hit Parked car during Night. We can also see there are alot of "Other" collision type of cases in late night.

In [ ]:
sns.set_palette(sns.color_palette("hot"))
tempdf = df[(df["COLLISIONTYPE"]=="Left Turn")|(df["COLLISIONTYPE"]=="Right Turn")]
fig, ax = plt.subplots(figsize=(7,7))
ax.pie(tempdf["COLLISIONTYPE"].value_counts(), textprops={'color':"white", 'fontsize': 14}, autopct='%1.0f%%')

ax.legend(tempdf["COLLISIONTYPE"],
          title="Collision Type (Turns)",
          loc="upper center",
          bbox_to_anchor=(1, 0, 0.5, 1))
# plt.savefig('./blog/24.png')
plt.show()


In [ ]:
((df[df["COLLISIONTYPE"]=="Left Turn"]["OBJECTID"].count() - df[df["COLLISIONTYPE"]=="Right Turn"]["OBJECTID"].count()) / df[(df["COLLISIONTYPE"]=="Right Turn")|(df["COLLISIONTYPE"]=="Left Turn")]["OBJECTID"].count()) * 100

There is ~65% more chance to encounter a collision while taking a Left turn than taking a Right turn

In [ ]:
df[df["COLLISIONTYPE"]=="Left Turn"].groupby("ADDRTYPE")["OBJECTID"].count()

In [ ]:
df[df["COLLISIONTYPE"]=="Right Turn"].groupby("ADDRTYPE")["OBJECTID"].count()

In [ ]:
sns.set_palette(sns.color_palette("Paired"))
x = pd.DataFrame(df[(df["COLLISIONTYPE"]=='Other')&(df["TIMEOFDAY"]=='Late Night')].groupby('SDOT_COLDESC')['OBJECTID'].count()).sort_values(by='OBJECTID', ascending=False)

In [ ]:
others_values = x["OBJECTID"].tail(12).sum()
x.drop(x.tail(12).index,inplace=True)
x = x.append(pd.DataFrame(data={"OBJECTID":others_values}, index=["Others Remaining"]))

In [ ]:
x.plot(kind='pie',subplots=True,figsize=(10,10), autopct='%1.0f%%',textprops={'color':"grey", 'fontsize': 14}, explode=(0.1,0.1,0,0,0,0))
plt.axes().set_facecolor("white")
plt.axes().get_legend().remove()
plt.title("Collision type distribution for 'Late Night'")
# plt.savefig('./blog/25.png')
plt.show()

Motor Vehicles tend to hit fixed objects more during late night time

# Collision Type

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.countplot(y=df['SDOT_COLDESC'],ax=ax,orient="h")

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
df['SDOT_COLDESC'].value_counts(normalize=True).head(5).plot.barh(title="% of accidents by collision type",ax=ax)
ax.set_xlabel('% of accidents', fontsize=18)
ax.tick_params(axis="x", labelsize=12)
ax.tick_params(axis="y", labelsize=12)
# ax.set_ylabel('Collision Count', fontsize=18)
# plt.title("Collision count by Month for under influenced driving collisions", fontsize=18)
# plt.savefig('./blog/26.png')


We can see ~40% of collisions are of motor vehicle striking another motor vehicle’s front end at an angle.


In [ ]:
df1 = df.copy()

df1["COLLISIONTYPE"].fillna("Unknown",inplace=True)
df1["SDOT_COLDESC"].fillna("Unknown",inplace=True)
fig = px.sunburst(
    data_frame=df1[~df1["TIMEOFDAY"].isnull()],
    path= ["TIMEOFDAY",'COLLISIONTYPE',"SDOT_COLDESC"], 
    color="TIMEOFDAY",
    color_discrete_sequence=px.colors.qualitative.Pastel,
    maxdepth=-1,                        
    branchvalues="total", 
    title="Breakdown of collision type by time",
)
fig.update_traces(textinfo='label+percent entry') # percent parent
fig.update_layout(margin=dict(t=30, b=10, r=10, l=10),
                  width=1000, height=1000)
fig.show()

In [ ]:
pd.crosstab(df.COLLISIONTYPE, df.TIMEOFDAY).apply(lambda r: ((r/r.sum())*100).round(2), axis=1)

# Location wise analysis

In [ ]:
f_df = df.groupby('ADDRTYPE')[['FATALITIES']].sum()
f_df['INJURIES'] = df.groupby('ADDRTYPE')[['INJURIES']].sum()
f_df['SERIOUSINJURIES'] = df.groupby('ADDRTYPE')[['SERIOUSINJURIES']].sum()
f_df['ACDNTCOUNT'] = df.groupby('ADDRTYPE')[['OBJECTID']].count()
f_df = f_df.reset_index()
f_df

In [ ]:
sns.set_palette('twilight_shifted_r')
a = df.groupby('ADDRTYPE')[['FATALITIES']].sum()
f_df1 = a.apply(lambda x : 100 * x/float(x.sum()))
a['P_INJURIES'] = df.groupby('ADDRTYPE')[['INJURIES']].sum()
f_df1 = a.apply(lambda x : 100 * x/float(x.sum()))
a['P_SERIOUSINJURIES'] = df.groupby('ADDRTYPE')[['SERIOUSINJURIES']].sum()
f_df1 = a.apply(lambda x : 100 * x/float(x.sum()))
a['P_ACDNTCOUNT'] = df.groupby('ADDRTYPE')[['OBJECTID']].sum()
f_df1 = a.apply(lambda x : 100 * x/float(x.sum()))
#f_df1 = f_df1.drop(columns=['FATALITIES', 'INJURIES'])
f_df1.reset_index(inplace=True)
f_df1.rename(columns={'FATALITIES':'P_FATALITIES'}, inplace=True)
f_df1

fig, ax = plt.subplots(figsize=(10,10))
df2=pd.melt(f_df1,id_vars=['ADDRTYPE'],var_name='DESC', value_name='PERCENTAGE OF TOTAL')
df2
sns.barplot(x='ADDRTYPE', y='PERCENTAGE OF TOTAL', hue='DESC', data=df2, ax=ax)
plt.xticks(rotation=90)
# plt.savefig('./blog/29.png')

In [ ]:
layout = go.Layout(
    title="Location wise Type of Injuries",
    xaxis=dict(
        title="ACCIDENT ADDRESS TYPE",
        linecolor='#A9A9A9',
        showgrid=True),
    yaxis=dict(
        title="NUMBER OF ACCIDENTS",
        showgrid=False),
    barmode='group'
)

fig = go.Figure(data=[
    go.Bar(name='TOTAL #OF ACCIDENT', x=f_df['ADDRTYPE'], y=f_df['ACDNTCOUNT']),
    go.Bar(name='INJURIES', x=f_df['ADDRTYPE'], y=f_df['INJURIES']),
    go.Bar(name='SERIOUSINJURIES', x=f_df['ADDRTYPE'], y=f_df['SERIOUSINJURIES']),
    go.Bar(name='FATALITIES', x=f_df['ADDRTYPE'], y=f_df['FATALITIES'])
    
], layout= layout)
fig.show()

In [ ]:
pd.crosstab(df["COLLISIONTYPE"],df["ADDRTYPE"])

In [ ]:
df1 = df[df['INCDATE'].dt.year > 2016]
a= df1.groupby(['INTKEY','LOCATION'])['OBJECTID'].count() #,
a = a.to_frame()
a.reset_index(inplace=True)
a.rename(columns={'OBJECTID':'# OF ACCIDENTS'}, inplace=True)
bins = [0, 10, 20, 30, 40,50,120]
labels = ['Below 10', 'Between 10 and 20','Between 20 and 30','Between 30 and 40','Between 40 and 50',
         'Above 50']
a['BINNED'] = pd.cut(a['# OF ACCIDENTS'], bins, labels=labels)
a = a.sort_values(by=['# OF ACCIDENTS'],ascending=False)
b = a.head(20)
fig = px.bar(b, x="# OF ACCIDENTS", y="LOCATION", color="# OF ACCIDENTS")
fig.show()

Location "6th AVE AND JAMES ST" has abnormally high number of collisions since 2016

# Road and Lighting Conditions

In [ ]:
fig,ax = plt.subplots(nrows=2,ncols=1,figsize=(10,8))
sns.countplot(y=df['ROADCOND'], ax=ax[0],orient="h")
sns.countplot(y=df['LIGHTCOND'], ax=ax[1],orient="h")
# plt.savefig('./blog/33.png')

In [ ]:
df["LIGHTCOND"].unique()

In [ ]:
df[(df["SEVERITYCODE"]=="1")]["LIGHTCOND"].value_counts()

In [ ]:
fig, ax = plt.subplots(nrows=2,ncols=4,figsize=(20,10))
ax[0,0].pie(df[(df["SEVERITYCODE"]=="1")]["ROADCOND"].value_counts(), textprops={'color':"white", 'fontsize': 14})
ax[0,1].pie(df[(df["SEVERITYCODE"]=="2")]["ROADCOND"].value_counts(), textprops={'color':"white", 'fontsize': 14})
ax[0,2].pie(df[(df["SEVERITYCODE"]=="2b")]["ROADCOND"].value_counts(), textprops={'color':"white", 'fontsize': 14})
ax[0,3].pie(df[(df["SEVERITYCODE"]=="3")]["ROADCOND"].value_counts(), textprops={'color':"white", 'fontsize': 14})

ax[0,0].set_title("Property Damage Only Collision",color="grey")
ax[0,1].set_title("Injury Collision",color="grey")
ax[0,2].set_title("Serious Injury Collision",color="grey")
ax[0,3].set_title("Fatal Collision",color="grey")


ax[1,0].pie(df[(df["SEVERITYCODE"]=="1")]["LIGHTCOND"].value_counts(), textprops={'color':"white", 'fontsize': 14})
ax[1,1].pie(df[(df["SEVERITYCODE"]=="2")]["LIGHTCOND"].value_counts(), textprops={'color':"white", 'fontsize': 14})
ax[1,2].pie(df[(df["SEVERITYCODE"]=="2b")]["LIGHTCOND"].value_counts(), textprops={'color':"white", 'fontsize': 14})
ax[1,3].pie(df[(df["SEVERITYCODE"]=="3")]["LIGHTCOND"].value_counts(), textprops={'color':"white", 'fontsize': 14})


ax[0,3].legend(["Dry","Wet","Unknown","Ice","Snow/Slush","Other","Standing Water","Sand/Mud/Dirt","Oil"],
          title="Road Condition",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))
ax[1,3].legend(["Daylight","Dark-Street Light On","Unknown","Dawn","Dark-No Street Lights","Dark-Street Light Off","Other","Dark-Unknown Lighting"],
          title="Light Condition",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))
# plt.savefig('./blog/34.png')
plt.show()

Severe accidents are relatively less in daylight

# Effect of COVID-19 pandemic on vehicle collisions

In 2020, whole world faced a pandemic of the COVID-19 outbreak. Its effect on travel behaviour have also been sudden and wide ranging. 

In [ ]:
a = df[df['YEAR']!=2020].groupby(['MONTH'])['OBJECTID'].count()/df[df['YEAR']!=2020].groupby(['MONTH']).size().nunique()
a = a.to_frame()
a.reset_index(inplace=True)
a.rename(columns = {'OBJECTID':'2013-2019'}, inplace=True)
b= df[df['YEAR']==2020].groupby(['MONTH'])['OBJECTID'].count()
b = b.to_frame()
b.reset_index(inplace=True)
b.rename(columns = {'OBJECTID':'2020'}, inplace=True)
# merging dataframes
df_compare = pd.merge(a, b, on='MONTH', how='left')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_compare['MONTH'], y=df_compare['2013-2019'],mode='lines+markers', 
                         name='Before 2020'))
fig.add_trace(go.Scatter(x=df_compare['MONTH'], y=df_compare['2020'],mode='lines+markers',
                        name='2020'))
fig.update_layout(title="Impact of COVID-19 on Road Accidents",
                  xaxis_title='Month',
                  yaxis_title='Average Number of Accidents'
                 )
# plt.savefig('./blog/35.png')
fig.show()

We can see that there is a sudden drop in accident cases in 2020, due to COVID-19 pandemic

# Collisions on Map

In [ ]:
sf_map = folium.Map(location=[df["Y"].mean(), df["X"].mean()],  zoom_start=12, control_scale=True, min_zoom=11)

df_filtered = df[(df["YEAR"]>= 2014)]
HeatMap(data=df_filtered[["X","Y","VEHCOUNT"]].groupby(['Y', 'X']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(sf_map)

sf_map

We can see density of accident is greater in Central Seattle, around University Road and First Hill

# Conclusion
After the thorough analysis, we have a better understanding of the causes and reasons for major road accidents. We see that although only 26% of accidents involve any injury, the number is still too high. The majority of the cases happen between the afternoon and evening, thus people driving during these hours should be more careful. The visuals convey to us that severity of the accident is higher during the day than late night, something we thought wasn't likely. We also see that speeding causes more accidents late at night. Additionally, in night-time accidents primarily happen because of less visibility. Thus, drivers should avoid speeding and be more careful during the night to avoid accidents. The data tells us that severe accidents more likely happen at blocks or intersections, as expected.

Moreover, we see that there is a spike I drunken driving cases during December (holiday season), the law enforcement should particularly take care during this time for such incidents. Here we see that utilizing analysis on top of the visuals entails one to bring out insights that may lead to a breakthrough, like the chances of being in an accident while turning left are more than while turning right. This information forms the core of the data analysis process. An effective, well-designed, and beautiful visualization is great but you shouldn't lose the context hidden in data art and must focus upon data-driven decisions that may engage your stakeholders. It is crucial to be a good visual storyteller rather than a visual artist, bringing out stories from data is what companies seek and require.